# Week1_Library 과제

### Q1. Library 와 Framework 의 차이 간단하게 서술하시오. (100자 내외)

* Library : 특정한 기능을 구현하기 위해 만들어진 함수들의 집합, 필요할 때 자유롭게 사용할 수 있다.
    *   Ex) matplotlib(시각화 라이브러리), BeautifulSoup(HTML파싱 라이브러리)
* Framework : 특정 운영 체제를 위한 응용 프로그램를 구현하는 클래스와 라이브러리 모임 여러개의 재사용 가능한 모듈과 패키지를 모아둔 코드 집합체
    *   Ex) django, Web2Py
* Library는 사용자가 필요할 때 자유롭게 쓰지만, Framework는 정해둔 틀 안에 필요한 코드를 작성하는 것이다. 

### Q2. 딥러닝과 머신러닝의 관계 및 특징, 차이 간단하게 서술하시오. (200자 내외)

머신러닝은 기계학습이라고도 불리우며, 주어진 데이터를 통해 스스로 학습하는 것을 말한다. 빅데이터를 분석하고 가공해서 새로운 정보를 얻고 미래를 예측하는 기술    
딥러닝은 머신러닝의 하위 개념으로 인간의 신경망을 따라 만든 인공신경망을 통해 인공지능을 만드는 것이다.

### Q3. 아래의 코드에 주석 달기.

In [3]:
# pytorch : facebook에서 제공한 딥러닝 도구
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms

# torch.cuda설치 여부 확인
# 신경망은 CPU보다 GPU사용을 권장하나 아직 cuda설치 전이라 cpu만 사용가능
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(45)
if device == 'cuda':
    torch.cuda.manual_seed_all(45)
print(device + " is available")

cpu is available


In [3]:
learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5

In [4]:

# torch에서 제공하는 데이터셋 중
# MNIST 데이터 셋 로드(훈련데이터) 
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([
        # 데이터의 범위를 0~255에서 0~1사이 값으로 변환
        transfroms.ToTensor() 
    ])
)

# MNIST 데이터 셋 로드(테스트데이터) 
test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST\MNIST\raw\train-images-idx3-ubyte.gz to ./data/MNIST\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST\MNIST\raw\train-labels-idx1-ubyte.gz to ./data/MNIST\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data/MNIST\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data/MNIST\MNIST\raw



In [5]:
# dataloader를 이용 MNIST 데이터셋 로딩
# train, test loader생성
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

# input size를 알기 위해서
examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

torch.Size([1, 28, 28])

In [6]:

# Convolutional Neural Networks
class ConvNet(nn.Module):
  def __init__(self):                                 # layer정의
        super(ConvNet, self).__init__() 
        
        # conv : 이미지 특징을 추출하는 층
        # input size = 28 x 28
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)  
        # input channel = 1, filter = 10, kernel size = 5, zero padding = 0, stribe = 1
        # ((W-K+2P)/S)+1 공식으로 ((28-5+0)/1)+1=24 -> 24x24로 변환
        # maxpooling하면 12x12

        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) 
        # input channel = 1, filter = 10, kernel size = 5, zero padding = 0, stribe = 1
        # ((12-5+0)/1)+1=8 -> 8x8로 변환
        # maxpooling하면 4x4

        # 랜덤하게 뉴런을 종료해서 학습을 방해해 학습용 데이터에 치우치는 현상을 막는다.
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) 
        # 오버피팅을 방지하고, 연산에 들어가는 자원을 줄이기 위해 출력 값에서 일부만 취해 사이즈가 작은 이미지를 만든다. maxpolling
        self.mp = nn.MaxPool2d(2)
        # 4x4x20 vector로 flat한 것을 100개의 출력으로 변경
        self.fc1 = nn.Linear(320,100) 
        # 100개의 출력을 10개의 출력으로 변경
        self.fc2 = nn.Linear(100,10) 

      # 순방향
  def forward(self, x):
        # 활성화 함수로 쓰이는 ReLU
        # convolution layer 1번에 relu를 씌우고 maxpool, 결과값은 12x12x10
        x = F.relu(self.mp(self.conv1(x))) 
        # convolution layer 2번에 relu를 씌우고 maxpool, 결과값은 4x4x20
        x = F.relu(self.mp(self.conv2(x))) 
        x = self.drop2D(x) 
        x = x.view(x.size(0), -1)          # flat
        x = self.fc1(x)                    # fc1 레이어에 삽입
        x = self.fc2(x)                    # fc2 레이어에 삽입
        # fully-connected layer에 넣고 logsoftmax 적용
        return F.log_softmax(x)

In [7]:

# CNN 객체 생성
model = ConvNet().to(device) 
# 손실함수와 옵티마이저 선택
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [8]:

for epoch in range(epochs):                     # epochs수 만큼 반복 
    avg_cost = 0
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()                   # 모든 model의 gradient 값을 0으로 설정
        hypothesis = model(data)                # 모델을 forward pass해 결과값 저장
        cost = criterion(hypothesis, target)    # output, target의 손실 계산
        cost.backward()                         # backward 함수를 호출해 gradient갱신
        optimizer.step()                        # 모델의 학습 파라미터 갱신
        avg_cost += cost / len(train_loader)    # loss값을 변수에 누적하고 train_loader의 개수로 나눔 = 평균
    print('[Epoch: {:>4}]  cost = {:>.9}'.format(epoch + 1, avg_cost))

C:\Users\lbhde\AppData\Local\Temp\ipykernel_35952\3803583406.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


[Epoch:    1]  cost = 0.329685748
[Epoch:    2]  cost = 0.111265108
[Epoch:    3]  cost = 0.084695518
[Epoch:    4]  cost = 0.0729407147
[Epoch:    5]  cost = 0.0648430437


In [ ]:

# test
model.eval()            # evaluate mode로 전환 dropout 이나 batch_normalization 해제 
with torch.no_grad():   # grad 해제 
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1]           # 출력이 분류 각각에 대한 값으로 나타나기 때문에, 가장 높은 값을 갖는 인덱스를 추출
        total += len(target)                        # 전체 클래스 개수 
        correct += (preds==target).sum().item()     # 예측값과 실제값이 같은지 비교
        
    print('Test Accuracy: ', 100.*correct/total, '%')
     

## 첫 정규세션 들으시느라 고생 많으셨습니다.